In [1]:
import torch
import matplotlib.pyplot as pyplot
import numpy as np
import pandas as pd
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from tqdm.auto import tqdm
import torchvision.models as models
import cv2

c:\Users\HP\Desktop\phone-detector\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# hyperparams

batch_size = 32 
device = "cuda" if torch.cuda.is_available() else "cpu"
epochs = 3
lr = 1e-5

In [3]:
imagenet_normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.RandomAffine(degrees=30, translate=(0.1, 0.1)),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.ToTensor(),
    imagenet_normalize
])


test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    imagenet_normalize
])


In [4]:
model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)

for param in model.features[-5:].parameters():
    param.requires_grad = True



model.classifier = nn.Sequential(
    nn.Linear(25088, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 1)  
)

model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [6]:
def train_loop(epochs, loader, model, loss_fn, optimizer):
    total_batches = len(loader)

    for epoch in tqdm(range(epochs)):

        total_epoch_loss = 0
        print(f"\n -------- Epoch {epoch} --------")
        
        for batch, (X, Y) in enumerate(loader):
            model.train()

            X, Y = X.to(device), Y.to(device)

            Y_pred = model(X)
            Y = Y.unsqueeze(1).float()
            loss = loss_fn(Y_pred, Y)
            total_epoch_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


        total_epoch_loss /= total_batches
        print()
        print(f"Avg training loss: {total_epoch_loss:.2f}")


def test_loop(loader, model, loss_fn):
    
    total_batches = len(loader)

    model.eval()
    total_loss, total_acc = 0, 0
    with torch.inference_mode():

        for X, Y in loader:
            
            X, Y = X.to(device), Y.to(device)

            Y_pred = model(X)
            Y = Y.unsqueeze(1).float()
            loss = loss_fn(Y_pred, Y)
            total_loss += loss.item()

            probs = torch.sigmoid(Y_pred)
            preds = (probs > 0.5).float()
            correct = (preds == Y).sum().item()
            total_acc += correct

        total_loss /= total_batches
        total_acc = total_acc / len(loader.dataset)

    return total_acc
        


In [7]:
train_dataset = datasets.ImageFolder(root=r"C:\Users\HP\Desktop\phone-detector\Train-6", transform=train_transforms)
test_dataset  = datasets.ImageFolder(root=r"C:\Users\HP\Desktop\phone-detector\Test-2",  transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [ ]:
from ray import tune
from ray.tune.search.optuna import OptunaSearch
from ray.tune import TuneConfig

def custom_trial_dir_name(trial):
    return f"objective_lr{trial.config['lr']:.4f}_d{trial.config['dropout']}_f{trial.config['features']}"


def objective(config): 
    model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1) 
    
    for param in model.features[config['features']:].parameters():
        param.requires_grad = True


    model.classifier = nn.Sequential(
        nn.Linear(25088, 512),
        nn.ReLU(),
        nn.Dropout(config['dropout']),
        nn.Linear(512, 1)  
    )

    model.to(device)

    optimizer = torch.optim.AdamW([
        {"params": model.features[config['features_to_train']:].parameters(), "lr": config['lr']},  
        {"params": model.classifier.parameters(), "lr": config['clr']},     
    ], weight_decay=config['weight_decay'])

    loss_fn = torch.nn.BCEWithLogitsLoss()


    train_loop(config['epochs'], train_loader, model, loss_fn, optimizer)
    acc = test_loop(test_loader, model, loss_fn)
    tune.report({"mean_accuracy": acc})  


search_space = {"lr": tune.loguniform(1e-5,  1e-3),  
                "dropout" : tune.choice([0.2, 0.3, 0.4, 0.5]),
                "clr" : tune.loguniform(1e-5,  1e-3),
                "weight_decay" : tune.loguniform(1e-5,  1e-3),
                "features" : tune.choice([-2, -3, -4, -5, -6, -7]),
                "features_to_train" : tune.choice([-2, -3, -4, -5, -6, -7]),
                "epochs" : tune.choice([2, 4, 6, 8, 10])
                }

algo = OptunaSearch() 

tuner = tune.Tuner(  
    objective,
    tune_config=tune.TuneConfig(
        metric="mean_accuracy",
        mode="max",
        search_alg=algo,
        trial_dirname_creator=custom_trial_dir_name
    ),
    run_config=tune.RunConfig(
        stop={"training_iteration": 1},
    ),
    param_space=search_space,
)
results = tuner.fit()
print("Best config is:", results.get_best_result().config)

  0%|          | 0/8 [00:00<?, ?it/s]


(objective pid=8396) 
(objective pid=8396)  -------- Epoch 0 --------


(pid=gcs_server) [2025-12-23 17:28:58,475 E 20672 24048] (gcs_server.exe) gcs_server.cc:302: Failed to establish connection to the event+metrics exporter agent. Events and metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14
(raylet) [2025-12-23 17:29:03,887 E 10536 13628] (raylet.exe) main.cc:975: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14
 12%|█▎        | 1/8 [03:31<24:43, 211.97s/it]


(objective pid=8396) 
(objective pid=8396) Avg training loss: 0.46
(objective pid=8396) 
(objective pid=8396)  -------- Epoch 1 --------


 25%|██▌       | 2/8 [06:57<20:48, 208.04s/it]


...

(objective pid=8396) 
(objective pid=8396) Avg training loss: 0.08
(objective pid=8396) 
(objective pid=8396)  -------- Epoch 7 --------


100%|██████████| 8/8 [27:46<00:00, 208.26s/it]


(objective pid=8396) 
(objective pid=8396) Avg training loss: 0.08


2025-12-23 17:57:01,320	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to 'C:/Users/HP/ray_results/objective_2025-12-23_17-28-24' in 0.0202s.
2025-12-23 17:57:01,343	INFO tune.py:1041 -- Total run time: 1707.33 seconds (1707.12 seconds for the tuning loop).


Best config is: {'lr': 1.9939377918517657e-05, 'dropout': 0.3, 'clr': 0.00031384783419999377, 'weight_decay': 3.2730287919730626e-05, 'features': -7, 'features_to_train': -4, 'epochs': 8}


In [9]:
def live_pred(model):
    cam = cv2.VideoCapture(0)

    while True:

        ret, frame = cam.read()
        if not ret:
            return

        frame = cv2.resize(frame, (224, 224))

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        tensor = torch.from_numpy(rgb_frame).permute(2, 0, 1).unsqueeze(0).float() / 255.0
        tensor = tensor.to(device)

        mean = torch.tensor([0.485, 0.456, 0.406], device=device).view(1,3,1,1)
        std = torch.tensor([0.229, 0.224, 0.225], device=device).view(1,3,1,1)
        tensor = (tensor - mean) / std


        with torch.no_grad():
            pred = model(tensor)


        print(f"Prediciton: {1 if pred.item() >= 0.5 else 0}")
        if cv2.waitKey(1) & 0XFF == ord('q'):
            break

        cv2.imshow("Live feed", frame)

    cam.release()
    cv2.destroyAllWindows()
    return 